In [1]:
import pandas as pd
from os import scandir

In [7]:
name_files = scandir("names")
df = pd.DataFrame()
i = 0
for file in name_files:
    year = file.name[3:7]
    with open(f"names/{file.name}", "r") as handle:
        lines = handle.read().splitlines()
    names = [[year] + line.split(",") for line in lines]
    df = df.join(pd.DataFrame(names, columns=["year", "name", "gender", "populairity"]))
    i += 1
    if i == 2:
        break
df


ValueError: columns overlap but no suffix specified: Index(['year', 'name', 'gender', 'populairity'], dtype='object')

In [6]:
df

""
